<a href="https://colab.research.google.com/github/akshaykadidal/akshayakadidal.blogspot/blob/master/Keras_nn_limitvalues.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I have always had problems predicting continious values. When we predict a value that cannot be negative the model would predict negative value. Now with the nn architecture we have a solution to the problem

1. standardize your y_train with min max standardization (set your own min max if you like)
2. build you nn. In the output layer select sigmoid activation with mape or rmse as loss function. 
3. Convert your prediction to actual scale. This will ensure that the values that you are predicting will always fall between the min and max values determined by you

Let's try this with Titanic data set to predict age of the passengers

The Titanic data is sourced from Kaggle.com 


In [0]:
!pip install distance

In [0]:
!pip install StringDist

  Created wheel for StringDist: filename=StringDist-1.0.9-cp36-cp36m-linux_x86_64.whl size=24376 sha256=256644e55a50f25b75b53fe78225717975a3332d815e23cb92495b7194ab6edd
  Stored in directory: /root/.cache/pip/wheels/c3/1b/bb/bf0de4d64d8ca38759811fe3353e441f12feb606fb3d1d2e11
Successfully built StringDist


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
import io
from google.colab import files
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
import sklearn.cluster
import distance
import stringdist

In [0]:
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn import metrics 
from scipy.spatial.distance import cdist
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as shc

In [0]:
upload = files.upload()

In [0]:
df1 = pd.read_csv(io.BytesIO(upload['train.csv']))

KeyError: ignored

In [0]:
upload = files.upload()

In [0]:
df2 =  pd.read_csv(io.BytesIO(upload['test.csv']))

In [0]:
data = pd.concat([df1,df2], axis = 0)

In [0]:
data.head()

In [0]:
print (data.shape, 'data')
print (df1.shape, 'df1')
print (df2.shape, ' df2')

In [0]:
data.Age.describe()

In [0]:
%matplotlib inline

In [0]:
hist = data.Age.hist()
hist.plot()
hist_m = data[data['Sex']=='male'].Age.hist()
hist_f = data[data['Sex']=='female'].Age.hist()
plt.show()

Creating some feature engineered columns
Looks like people who bought tickets together have one ticket number / id. Their Fare is also based on number of tickets sold (or so it appears)

In [0]:
data['GroupSize'] = data.groupby('Ticket')['Ticket'].transform('count')

In [0]:
data['Istoddler'] = (data['Age']<4)*1
data['IsChild'] = ((data['Age']>=4) & (data['Age']< 13))*1
data['IsYoung'] = ((data['Age']>=13) & (data['Age']< 18))*1

In [0]:
data['ToddlersPergroup'] = data.groupby('Ticket')['Istoddler'].transform('sum')
data['ChildrenPergroup'] = data.groupby('Ticket')['IsChild'].transform('sum')
data['Youngpergroup'] = data.groupby('Ticket')['IsYoung'].transform('sum')

In [0]:
data[data['Fare'].isna()]

In [0]:
missing_fare = data[data['GroupSize']==1 & (data['Pclass']==3) & ( data['Embarked']=='S')].Fare.mode()

In [0]:
data['Fare'].fillna(missing_fare[0], inplace = True)

In [0]:
data[data['Fare'].isna()]

In [0]:
x1 = pd.to_numeric(data[data['Ticket'].str.isnumeric()].Ticket).unique()
x2 = x1.reshape(-1, 1)

In [0]:
X = np.array(list(zip(x1, x2))).reshape(len(x1), 2)

In [0]:
distortions = [] 
inertias = [] 
mapping1 = {} 
mapping2 = {} 
K = range(1,10) 
  
for k in K: 
    #Building and fitting the model 
    kmeanModel = KMeans(n_clusters=k).fit(x2) 
    kmeanModel.fit(X)     
      
    distortions.append(sum(np.min(cdist(X, kmeanModel.cluster_centers_, 
                      'euclidean'),axis=1)) / X.shape[0]) 
    inertias.append(kmeanModel.inertia_) 
  
    mapping1[k] = sum(np.min(cdist(X, kmeanModel.cluster_centers_, 
                 'euclidean'),axis=1)) / X.shape[0] 
    mapping2[k] = kmeanModel.inertia_ 

In [0]:
plt.plot(K, distortions) 
plt.xlabel('Values of K') 
plt.ylabel('Distortion') 
plt.title('The Elbow Method using Distortion') 
plt.show() 

In [0]:
kmeanModel = KMeans(n_clusters=5, random_state=23).fit(x2) 
kmeanModel.fit(x2) 

In [0]:
clstr = {'T_Cluster':kmeanModel.labels_, 'Ticket':x2.reshape(705,)}

In [0]:
cluster1 = pd.DataFrame(clstr)

In [0]:
cluster1.T_Cluster = cluster1.T_Cluster.astype('category')

In [0]:
cluster1.groupby('T_Cluster').describe()

In [0]:
x1 =data[-data['Ticket'].str.isnumeric()].Ticket.unique()
x = data[-data['Ticket'].str.isnumeric()].Ticket.str.replace(r'\W+','').unique()

In [0]:
lev_similarity = -1*np.array([[distance.levenshtein(x1,x2) for x1 in x] for x2 in x]) #dendo gramss wants a negetive matrix The symmetric non-negative hollow observation matrix looks suspiciously like

In [0]:
lev_similarity

In [0]:
plt.title("Dendograms")
dend = shc.dendrogram(shc.linkage(lev_similarity, method='ward'))

In [0]:
model = AgglomerativeClustering(distance_threshold=None, n_clusters=5)

In [0]:
model = model.fit(lev_similarity)

In [0]:
clstr = {'T_Cluster':model.labels_, 'Ticket':x1.reshape(224 ,)}

In [0]:
cluster2 = pd.DataFrame(clstr)

In [0]:
cluster2.T_Cluster = cluster2.T_Cluster + 4

In [0]:
cluster2.T_Cluster = cluster2.T_Cluster.astype('category')

In [0]:
cluster2.sort_values('Ticket')

In [0]:
T_Cluster = pd.concat([cluster1,cluster2], axis=0)

In [0]:
T_cluster.Ticket = T_cluster.Ticket.astype('str')

In [0]:
data  = data.merge(T_cluster, on= 'Ticket', how='left')

In [0]:
data.Pclass = data.Pclass.astype('category')
data.Sex = data.Sex.astype('category')
data.Cabin = data.Cabin.astype('category')
data.Embarked = data.Embarked.astype('category')
#data.Survived = data.Survived.astype('bool')
data.Istoddler = data.Istoddler.astype('bool')
data.IsChild = data.IsChild.astype('bool')
data.IsYoung = data.IsYoung.astype('bool')
data.T_Cluster = data.T_Cluster.astype('category')


In [0]:
data['Family_Size']= data.SibSp + data.Parch

In [0]:
'''
model  = DecisionTreeRegressor(random_state=0,criterion='mse')
X = pd.get_dummies(data[['Fare','Embarked','GroupSize','Age','Istoddler','IsChild','IsYoung','ToddlersPergroup','ChildrenPergroup','Youngpergroup','T_Cluster']].dropna(),drop_first=True)
y = X['Fare']
model = model.fit(X, y)
'''

Using the standard way predicting Age

In [0]:
def get_model(activation, optimizer,batch_size):
  model = Sequential()
  model.add(Dense(32, input_shape = (i_shp,), activation = activation))
  model.add(BatchNormalization())
  model.add(Dense(32, activation=activation))
  model.add(BatchNormalization())
  model.add(Dense(1))
  model.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics=['mse'])
  return(model)


In [0]:
params = {'activation': ['relu', 'tanh'], 'batch_size': [32, 128, 256], 
          'epochs': [50, 100, 200], 'optimizer' : ['adam','sgd']}

In [0]:
model = KerasRegressor(build_fn=get_model, batch_size=16)

In [0]:
random_search = RandomizedSearchCV(model, param_distributions = params, cv = 3)

In [0]:
X_train = data[['Survived','Pclass','Fare','Age','Embarked','GroupSize','Istoddler','IsChild','IsYoung','ToddlersPergroup','ChildrenPergroup','Youngpergroup','T_Cluster','Family_Size']].dropna()

In [0]:
X = pd.get_dummies(X_train.drop('Age', axis=1), drop_first=True)
Y = X_train['Age']
i_shp = len(X.columns.values)
X.Survived =  X.Survived.astype('int')
X.Istoddler =  X.Istoddler.astype('int')
X.IsChild =  X.IsChild.astype('int')
X.IsYoung =  X.IsYoung.astype('int')

In [0]:
#random_search.fit(X,y)

In [0]:
random_search.best_params_

In [0]:
random_search.best_estimator_.predict(X).min()

You will see that we have negative values in ages which are not correct. so let's take the proposed approach.

In [0]:
from sklearn.preprocessing import MinMaxScaler

In [0]:
type(y.values)

In [0]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
y = min_max_scaler.fit_transform(np.float32(Y[:, np.newaxis]))

In [0]:
def get_model():
  model = Sequential()
  model.add(Dense(32, input_shape = (i_shp,), activation='tanh'))
  model.add(BatchNormalization())
  model.add(Dense(32, activation = 'tanh'))
  model.add(BatchNormalization())
  model.add(Dense(1, activation = 'sigmoid'))
  model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=['mse'])
  return(model)

In [0]:
model = get_model()

In [0]:
model.fit(X , y)

In [0]:
y_pred = model.predict(X)

In [0]:
(scaler.inverse_transform(y_pred.T) - Y.values).max() # we have a model that predicts values with a presion of less than a year

In [0]:
Y